In [ ]:
pip install PyMuPDF transformers torch openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 25.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x8

In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import openai
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
from datasets import load_metric

In [ ]:
# Step 1: Extract and Preprocess Text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [ ]:
def preprocess_text(text):
    # Remove unnecessary newline characters
    text = re.sub(r'\n+', ' ', text)  # Replace multiple newlines with a space
    paragraphs = text.split('. ')  # Split text into sentences
    cleaned_paragraphs = [para.strip() + '.' for para in paragraphs if para.strip()]
    return cleaned_paragraphs


In [ ]:
pdf_path = "/content/Test_for_bel.pdf"
book_text = extract_text_from_pdf(pdf_path)
cleaned_paragraphs = preprocess_text(book_text)

print(cleaned_paragraphs)

['What is object-oriented programming (OOP)? Object-oriented programming (OOP) is a computer programming model that organizes software design around data, or objects, rather than functions and logic.', 'An object can be defined as a data field that has unique attributes and behavior.', 'OOP focuses on the objects that developers want to manipulate rather than the logic required to manipulate them.', 'This approach to programming is well suited for software that is large, complex and actively updated or maintained.', 'This includes programs for manufacturing and design, as well as mobile applications.', 'For example, OOP can be used for manufacturing system simulation software.', 'The organization of an object-oriented program also makes the method beneficial for collaborative development, where projects are divided into groups.', 'Additional benefits of OOP include code reusability, scalability and efficiency.', 'The first step in OOP is to collect all of the objects a programmer wants

In [ ]:
print(f"Number of paragraphs extracted: {len(cleaned_paragraphs)}")
print("Sample paragraphs:")
for i, paragraph in enumerate(cleaned_paragraphs[:50]):
    print(f"Paragraph {i+1}: {paragraph}")

Number of paragraphs extracted: 109
Sample paragraphs:
Paragraph 1: What is object-oriented programming (OOP)? Object-oriented programming (OOP) is a computer programming model that organizes software design around data, or objects, rather than functions and logic.
Paragraph 2: An object can be defined as a data field that has unique attributes and behavior.
Paragraph 3: OOP focuses on the objects that developers want to manipulate rather than the logic required to manipulate them.
Paragraph 4: This approach to programming is well suited for software that is large, complex and actively updated or maintained.
Paragraph 5: This includes programs for manufacturing and design, as well as mobile applications.
Paragraph 6: For example, OOP can be used for manufacturing system simulation software.
Paragraph 7: The organization of an object-oriented program also makes the method beneficial for collaborative development, where projects are divided into groups.
Paragraph 8: Additional benefits o

In [ ]:
# Step 2: Understand and Encode Text using BERT
bert_model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def encode_paragraphs(paragraphs, tokenizer, model):
    encoded_paragraphs = []
    for paragraph in paragraphs:
        # Tokenize the paragraph
        inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, padding=True, max_length=512)
        # Pass the tokens through the BERT model
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract the hidden states from the outputs
        hidden_states = outputs.last_hidden_state
        # Take the mean of the hidden states to get a single vector representation for the paragraph
        paragraph_embedding = hidden_states.mean(dim=1)
        # Append the embedding to the list
        encoded_paragraphs.append(paragraph_embedding.squeeze().numpy())
    return encoded_paragraphs

encoded_paragraphs = encode_paragraphs(cleaned_paragraphs, bert_tokenizer, bert_model)


In [ ]:
# encoded_paragraphs = encode_paragraphs(cleaned_paragraphs)

In [ ]:
# add key here


In [ ]:
import time
def generate_questions(paragraph, max_retries=5, wait_time=2):
    retries = 0
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Generate five meaningful questions based on the following paragraph:\n\n{paragraph}\n\nQuestions:"}
                ],
                max_tokens=150,
                n=1,
                stop=None
            )
            questions = response['choices'][0]['message']['content'].strip().split('\n')
            return questions
        except openai.error.RateLimitError:
            retries += 1
            time.sleep(wait_time)
            wait_time *= 2
        except openai.error.APIError as e:
            print(f"API Error: {e}")
            return []
        except openai.error.APIConnectionError as e:
            print(f"API Connection Error: {e}")
            retries += 1
            time.sleep(wait_time)
            wait_time *= 2
        except openai.error.InvalidRequestError as e:
            print(f"Invalid Request: {e}")
            return []
        except openai.error.AuthenticationError as e:
            print(f"Authentication Error: {e}")
            return []
        except openai.error.PermissionError as e:
            print(f"Permission Error: {e}")
            return []
        except Exception as e:
            print(f"Unexpected Error: {e}")
            return []
    return []

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import download
from nltk.tag import pos_tag
import nltk
nltk.download('averaged_perceptron_tagger')
download('punkt')
def generate_question(paragraph):
    sentences = sent_tokenize(paragraph)
    questions = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        pos_tags = pos_tag(words)

        # Basic heuristic to identify potential question subjects (nouns, pronouns)
        subject = None
        verb = None
        for word, tag in pos_tags:
            if tag in ['NN', 'NNS', 'NNP', 'NNPS']:  # Nouns
                subject = word
            elif tag.startswith('VB'):  # Verbs
                verb = word
            if subject and verb:
                break

        if subject and verb:
            question = f"What {verb} {subject}?"
            questions.append(question)

    return questions

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

import concurrent.futures
paragraphs_with_questions = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_paragraph = {executor.submit(generate_question, paragraph): paragraph for paragraph in cleaned_paragraphs}
    for future in concurrent.futures.as_completed(future_to_paragraph):
        paragraph = future_to_paragraph[future]
        try:
            questions = future.result()
            paragraphs_with_questions.append((paragraph, questions))
        except Exception as exc:
            print(f"Generated an exception: {exc}")
            paragraphs_with_questions.append((paragraph, []))

# Optional: Save the results to a DataFrame for further analysis
df = pd.DataFrame(paragraphs_with_questions, columns=['Paragraph', 'Questions'])
df.to_csv('paragraphs_with_questions.csv', index=False)

In [ ]:
print("Sample entries from paragraphs_with_questions:")
for i, (paragraph, questions) in enumerate(paragraphs_with_questions):
    print(f"Paragraph {i+1}: {paragraph}")
    print(f"Questions: {questions}")

Sample entries from paragraphs_with_questions:
Paragraph 1: OOP focuses on the objects that developers want to manipulate rather than the logic required to manipulate them.
Questions: ['What focuses OOP?']
Paragraph 2: For example, OOP can be used for manufacturing system simulation software.
Questions: ['What be OOP?']
Paragraph 3: The organization of an object-oriented program also makes the method beneficial for collaborative development, where projects are divided into groups.
Questions: ['What makes program?']
Paragraph 4: Additional benefits of OOP include code reusability, scalability and efficiency.
Questions: ['What include OOP?']
Paragraph 5: What is object-oriented programming (OOP)? Object-oriented programming (OOP) is a computer programming model that organizes software design around data, or objects, rather than functions and logic.
Questions: ['What is programming?', 'What is OOP?']
Paragraph 6: Examples of an object can range from physical entities, such as a human bein

In [ ]:
print(paragraphs_with_questions)

[('OOP focuses on the objects that developers want to manipulate rather than the logic required to manipulate them.', ['What focuses OOP?']), ('For example, OOP can be used for manufacturing system simulation software.', ['What be OOP?']), ('The organization of an object-oriented program also makes the method beneficial for collaborative development, where projects are divided into groups.', ['What makes program?']), ('Additional benefits of OOP include code reusability, scalability and efficiency.', ['What include OOP?']), ('What is object-oriented programming (OOP)? Object-oriented programming (OOP) is a computer programming model that organizes software design around data, or objects, rather than functions and logic.', ['What is programming?', 'What is OOP?']), ('Examples of an object can range from physical entities, such as a human being who is described by properties like name and address, to small computer programs, such as widgets.', ['What range object?']), ('The first step in

In [ ]:
# Optional: Save the results to a DataFrame for further analysis
df = pd.DataFrame(paragraphs_with_questions, columns=['Paragraph', 'Questions'])
df.to_csv('paragraphs_with_questions.csv', index=False)

In [ ]:
data = {'paragraph': [], 'question': []}

In [ ]:
for paragraph, questions in paragraphs_with_questions:
    for question in questions:
        if question.strip():
            data['paragraph'].append(paragraph)
            data['question'].append(question.strip())

In [ ]:
df = pd.DataFrame(data)
df_train = df.sample(frac=0.8, random_state=42)
df_val = df.drop(df_train.index)

In [ ]:
print("Length of df:", len(df))
print("Length of df_train:", len(df_train))
print("Length of df_val:", len(df_val))

Length of df: 89
Length of df_train: 71
Length of df_val: 18


In [ ]:
# Step 4: Fine-Tune T5
model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pip install transformers[torch] accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.1 MB/s eta 0:00:00


In [ ]:


import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Define the dataset class
class ParagraphQuestionDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        inputs = self.tokenizer(row['paragraph'], max_length=512, truncation=True, padding='max_length', return_tensors='pt')
        targets = self.tokenizer(row['question'], max_length=64, truncation=True, padding='max_length', return_tensors='pt')
        inputs["labels"] = targets["input_ids"]
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        return inputs

# Example DataFrames (you need to replace these with your actual data)
df_train = pd.DataFrame({
    'paragraph': ["The cat sat on the mat.", "The dog barked loudly."],
    'question': ["Where did the cat sit?", "What did the dog do?"]
})

df_val = pd.DataFrame({
    'paragraph': ["The sun is shining brightly."],
    'question': ["How is the sun shining?"]
})

# Initialize the tokenizer and model
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Create the datasets
train_dataset = ParagraphQuestionDataset(df_train, t5_tokenizer)
val_dataset = ParagraphQuestionDataset(df_val, t5_tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create the Trainer instance
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,13.181464
2,No log,13.094467
3,No log,13.072628


TrainOutput(global_step=3, training_loss=13.0830078125, metrics={'train_runtime': 29.9208, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.1, 'total_flos': 812050808832.0, 'train_loss': 13.0830078125, 'epoch': 3.0})

In [ ]:
# Step 5: Generate Questions Using the Fine-Tuned T5 Model
def generate_question(paragraph):
    input_text = f"generate question: {paragraph}"
    input_ids = t5_tokenizer(input_text, return_tensors="pt").input_ids
    outputs = t5_model.generate(input_ids)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [ ]:
generated_questions = []

for paragraph in df_val['paragraph']:
    question = generate_question(paragraph)
    generated_questions.append(question)

df_val['generated_question'] = generated_questions


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=79910190831ee56aab49d88cc6126f73673f42605b877f274deace242ccda275
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

def tokenize_text(text, tokenizer):
    return tokenizer.tokenize(text)

# Prepare the data
references = [[tokenize_text(ref, t5_tokenizer)] for ref in df_val['question']]
predictions = [tokenize_text(pred, t5_tokenizer) for pred in df_val['generated_question']]

# Compute BLEU
bleu_score = bleu_metric.compute(predictions=predictions, references=references)

# Compute ROUGE
rouge_score = rouge_metric.compute(predictions=predictions, references=references)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)

NameError: name 'load_metric' is not defined